<a href="https://colab.research.google.com/github/hendrikyong/CVNL_Assignment_1/blob/main/CVNL_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.tokenize import word_tokenize
import requests
import json
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
# get json dataset from github
url = "https://raw.githubusercontent.com/snipsco/nlu-benchmark/master/2016-12-built-in-intents/benchmark_data.json"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Successfully fetched SNIPS dataset")

    # Extract intents and associated text queries
    snips_data = {}
    for domain in data["domains"]:
        for intent in domain["intents"]:
            intent_name = intent["benchmark"]["Snips"]["original_intent_name"]
            queries = [query["text"] for query in intent["queries"]]
            snips_data[intent_name] = queries

    print(f"Total intents: {len(snips_data)}")
    print("Example intent:", list(snips_data.keys())[0])
    print("Example queries:", snips_data[list(snips_data.keys())[0]][:5])
else:
    print("Failed to fetch SNIPS dataset")


Successfully fetched SNIPS dataset
Total intents: 10
Example intent: ShareCurrentLocation
Example queries: ["Share my location with Hillary's sister", 'Send my current location to my father', 'Share my current location with Jim', 'Send my location to my husband', 'Send my location']


In [10]:
all_words = [word for queries in snips_data.values() for query in queries for word in word_tokenize(query.lower())]

# build vocabulary
vocab = {word: idx + 2 for idx, word in enumerate(Counter(all_words))}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1

# map intent
intent_labels = {intent: idx for idx, intent in enumerate(snips_data.keys())}

def text_to_sequence(text, vocab, max_len=10):
    tokens = [vocab.get(word, vocab["<UNK>"]) for word in word_tokenize(text.lower())]
    return tokens[:max_len] + [vocab["<PAD>"]] * (max_len - len(tokens))

sample_text = "play some jazz music"
print(text_to_sequence(sample_text, vocab))


[1, 257, 212, 1, 0, 0, 0, 0, 0, 0]


In [11]:
class SNIPSDataset(Dataset):
    def __init__(self, data, vocab, intent_labels, max_len=10):
        self.data = data
        self.vocab = vocab
        self.intent_labels = intent_labels
        self.max_len = max_len
        self.samples = [(query, intent) for intent, queries in data.items() for query in queries]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        text, intent = self.samples[idx]
        tokenized_text = text_to_sequence(text, self.vocab, self.max_len)
        label = self.intent_labels[intent]
        return torch.tensor(tokenized_text), torch.tensor(label)

train_dataset = SNIPSDataset(snips_data, vocab, intent_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

sample_batch = next(iter(train_loader))
print("Sample batch text tensor shape:", sample_batch[0].shape)
print("Sample batch labels tensor shape:", sample_batch[1].shape)


Sample batch text tensor shape: torch.Size([32, 10])
Sample batch labels tensor shape: torch.Size([32])


In [12]:
class IntentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(IntentRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, embed_dim)
        _, hidden = self.rnn(x)  # hidden: (1, batch, hidden_dim)
        return self.fc(hidden.squeeze(0))  # Output (batch, output_dim)

# hyperparameters
vocab_size = len(vocab)
embed_dim = 64
hidden_dim = 128
output_dim = len(intent_labels)

model = IntentRNN(vocab_size, embed_dim, hidden_dim, output_dim)
print(model)


IntentRNN(
  (embedding): Embedding(649, 64, padding_idx=0)
  (rnn): RNN(64, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

# use adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# loss function
criterion = nn.CrossEntropyLoss()

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    # tqdm progress bar for batches
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for texts, labels in pbar:
            texts, labels = texts.to(device), labels.to(device)

            optimizer.zero_grad()  # Reset gradients
            outputs = model(texts)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model parameters
            total_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
            pbar.set_postfix(Loss=f"{loss.item():.4f}", Acc=f"{(correct/total)*100:.2f}%")

    avg_loss = total_loss / len(train_loader)
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}: Loss: {avg_loss:.4f}, Accuracy: {train_accuracy*100:.2f}%\n")


Using device: cuda


Epoch 1/10: 100%|██████████| 11/11 [00:00<00:00, 143.47batch/s, Acc=100.00%, Loss=0.0053]


Epoch 1: Loss: 0.0037, Accuracy: 100.00%



Epoch 2/10: 100%|██████████| 11/11 [00:00<00:00, 128.72batch/s, Acc=100.00%, Loss=0.0007]


Epoch 2: Loss: 0.0020, Accuracy: 100.00%



...

Epoch 8: Loss: 0.0047, Accuracy: 99.70%



Epoch 9/10: 100%|██████████| 11/11 [00:00<00:00, 114.36batch/s, Acc=100.00%, Loss=0.0014]


Epoch 9: Loss: 0.0078, Accuracy: 100.00%



Epoch 10/10: 100%|██████████| 11/11 [00:00<00:00, 113.21batch/s, Acc=99.70%, Loss=0.0029]

Epoch 10: Loss: 0.0204, Accuracy: 99.70%



In [26]:
from sklearn.metrics import accuracy_score

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model(texts)
        predictions = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f"Training Accuracy: {accuracy:.4f}")


Training Accuracy: 0.9970


In [28]:
def predict_intent(text, model, vocab, intent_labels):
    model.eval()
    tokenized = text_to_sequence(text, vocab)
    input_tensor = torch.tensor(tokenized).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        predicted_label = torch.argmax(output, dim=1).item()

    return [key for key, value in intent_labels.items() if value == predicted_label][0]

print(predict_intent("play some jazz music", model, vocab, intent_labels))
print(predict_intent("turn on the lights", model, vocab, intent_labels))
print(predict_intent("book a flight to New York", model, vocab, intent_labels))


GetWeather
RequestRide
RequestRide
